# Cas Kaggle: Spotify Music DataBase
## Anàlisi de les cançons presents a la base de dades de Spotify

## Objectius

Els objectius d'aquesta pràctica són:
* Entendre els diferents atributs de la base de dades.
* Visualitzar la correlació que els atributs tenen entre ells.
* Crear un model predictiu sobre la popularitat d'una cançó a partir d'altres atributs.

Primerament, començarem per carregar la nostra base de dades:

In [1]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
%matplotlib inline 
import scipy.stats

# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('../data/SpotifyFeatures.csv')
data = dataset.values

y = dataset['popularity']
x = dataset.drop(columns = ['popularity'])

print("Dimensionalitat de la BBDD:", dataset.shape)
print("Dimensionalitat de les entrades X", x.shape)
print("Dimensionalitat de l'atribut Y", y.shape)

Dimensionalitat de la BBDD: (232725, 18)
Dimensionalitat de les entrades X (232725, 17)
Dimensionalitat de l'atribut Y (232725,)


Un cop carregada, farem un primera comprovació per buscar si existeixen entrades amb algun valor NULL en la base de dades.

In [2]:
print("Número de valors no existents per cada atribut de la BBDD:")
print(dataset.isnull().sum())

Número de valors no existents per cada atribut de la BBDD:
genre               0
artist_name         0
track_name          0
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64


Ara que ja sebem que no tenim cap valor NULL en la base de dades, podem fer una primera visualització de quin format tenen les entrades referent les diferents cançons dins la base de dades i alguns estadístics dels nostres atributs numèrics: 

In [3]:
print("Exemple de 10 entrades aleatòries de la BBDD:")
dataset.sample(10)

Exemple de 10 entrades aleatòries de la BBDD:


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
10275,Alternative,Passion,Jesus Paid It All,73EIpn5AcedCWPOmxWMxH9,46,0.092,0.317,327400,0.562,0.000,B,0.893,-8.719,Major,0.034,76.915,3/4,0.218
46948,Blues,Kenny Wayne Shepherd,Woman Like You,2LAwa7amti4xehvCeWBRQ7,38,0.003,0.343,259720,0.901,0.000,G,0.258,-4.298,Major,0.044,174.134,4/4,0.668
54227,Movie,Debbie Davis,I'll Be Home for Christmas,30UE8yz5bht9criYIyyzpK,0,0.978,0.569,249560,0.154,0.008,D,0.126,-11.898,Major,0.029,92.870,4/4,0.231
92296,Hip-Hop,KirbLaGoop,I Can't Feel My Face (feat. Lil Peep & Mackned),2DciXUNQQYGCHP7eJmz3GD,51,0.170,0.863,195902,0.403,0.000,A,0.438,-8.401,Minor,0.335,139.952,4/4,0.238
133842,Reggae,G. Love & Special Sauce,Shooting Hoops,0gsFtv4YvS6DQieNmc9Rc3,32,0.267,0.712,208293,0.486,0.005,D,0.058,-6.973,Major,0.072,83.167,4/4,0.730
75410,Children's Music,Juice Music,Time Warp,4CXYoF0IDdToTd76obvja7,0,0.032,0.490,200093,0.662,0.001,A,0.054,-7.647,Major,0.038,171.866,4/4,0.515
184386,Movie,Randy Newman,I'm Dreaming,1MjalSfgbZc1jDw90roQ7I,14,0.967,0.585,191816,0.226,0.000,G,0.235,-12.054,Major,0.211,101.006,3/4,0.361
127504,Classical,Claude Debussy,6 Epigraphes antiques (version for solo piano)...,0M3Zx2PBvbmU1Sz0M4Qz62,32,0.993,0.276,151947,0.033,0.743,A,0.073,-28.471,Minor,0.039,65.515,5/4,0.043
226512,Soul,Erykah Badu,Love Of My Life (An Ode To Hip Hop) - Longer A...,0ieS2bue78yadTdPiyzsaO,50,0.134,0.618,337573,0.413,0.000,F,0.091,-9.311,Minor,0.398,87.412,4/4,0.462
63483,Folk,Steely Dan,Brooklyn (Owes The Charmer Under Me),5w6AGoRTmtlLK89lhrupBR,47,0.534,0.633,261133,0.475,0.000,F,0.159,-11.447,Major,0.024,105.442,4/4,0.643


In [4]:
print("Estadístiques dels atributs numèrics de la BD:")
dataset.describe()

Estadístiques dels atributs numèrics de la BD:


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000,232725.000
mean,41.128,0.369,0.554,235122.339,0.571,0.148,0.215,-9.570,0.121,117.667,0.455
std,18.190,0.355,0.186,118935.909,0.263,0.303,0.198,5.998,0.186,30.899,0.260
min,0.000,0.000,0.057,15387.000,0.000,0.000,0.010,-52.457,0.022,30.379,0.000
25%,29.000,0.038,0.435,182857.000,0.385,0.000,0.097,-11.771,0.037,92.959,0.237
50%,43.000,0.232,0.571,220427.000,0.605,0.000,0.128,-7.762,0.050,115.778,0.444
75%,55.000,0.722,0.692,265768.000,0.787,0.036,0.264,-5.501,0.105,139.054,0.660
max,100.000,0.996,0.989,5552917.000,0.999,0.999,1.000,3.744,0.967,242.903,1.000


A partir d'aquests estadístics, ja podem comprovar que en la base de dades tenim outliers, ja que veiem que la duració de la cançó més llarga és de 5552917 ms, que és igual a 92.5 minuts, i pel nostre cas no ens interesa, ja que pràcticament ningú l'escoltarà sencer d'una vegada. Per tant, he decidit eliminar aquelles entrades que tinguin duració superior a 10 minuts (900.000ms).

A més, també podem veure en el camp de loudness existeix com a mínim una entrada amb un valor per sobre de 0dB, que teòricament hauria de ser el valor màxim que una cançó podria prendre, per tant, també he eliminat aquelles entrades que el seu valor de loudness superi els 0dB.

In [5]:
dataset = dataset.drop(dataset[dataset['duration_ms'] > 900000].index)
dataset = dataset.drop(dataset[dataset['loudness'] > 0].index)
dataset.describe()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000,232008.000
mean,41.177,0.368,0.555,231755.332,0.571,0.148,0.215,-9.555,0.120,117.709,0.455
std,18.179,0.354,0.185,90095.906,0.263,0.302,0.198,5.984,0.185,30.895,0.260
min,0.000,0.000,0.057,15387.000,0.000,0.000,0.010,-52.457,0.022,30.379,0.000
25%,29.000,0.037,0.436,182726.000,0.386,0.000,0.097,-11.741,0.037,92.977,0.238
50%,43.000,0.231,0.571,220253.500,0.606,0.000,0.128,-7.753,0.050,115.879,0.445
75%,55.000,0.720,0.692,265267.000,0.787,0.035,0.263,-5.500,0.105,139.091,0.660
max,100.000,0.996,0.989,899933.000,0.999,0.999,1.000,0.000,0.967,242.903,1.000


Ara que ja hem tractat els outliers en atributs numèrics, ens toca tractar els atributs categòrics, en el nostre cas, ens tocarà passar els atributs key, mode i time_signature a valors numèrics.

Primer, treballarem amb el atribut key, on seguirem la seva posició en la distribució de classe de tons per a assignar-li un valor.

Les assignacions seran les següents:

| Tonalitat | Valor assignat |
|-----------|:-----------:|
| C | 0 |
| C# | 1 |
| D |  2 |
| D# | 3 |
| E |  4 |
| F | 5 |
| F# | 6 |
| G | 7 |
| G# | 8 |
| A | 9 |
| A# | 10 |
| B | 11 |

In [6]:
claus = dataset['key'].unique()
valors_claus = [1, 6, 0, 5, 7, 4, 3, 2, 10, 8, 9, 11]

for i in range(len(claus)):
    dataset.loc[dataset['key'] == claus[i], 'key'] = valors_claus[i]

Segon, canviarem els valors de mode, convertint els valors de mode Minor a 0 i els Major a 1:

In [7]:
modes = dataset['mode'].unique()
valors_modes = [1, 0]

for i in range(len(modes)):
    dataset.loc[dataset['mode'] == modes[i], 'mode'] = valors_modes[i]

Finalment, assignarem valors numèrics per a cada un dels diferents tipus de compàs disponibles, seguint el criteri següent:

| Compàs | Valor assignat |
|-----------|:-----------:|
| 0/4 | 0 |
| 1/4 | 1 |
| 3/4 | 2 |
| 4/4 | 3 |
| 5/4 | 4 |

In [8]:
compas = dataset['time_signature'].unique()
valors_compas = [3, 4, 2, 1, 0]

for i in range(len(compas)):
    dataset.loc[dataset['time_signature'] == compas[i], 'time_signature'] = valors_compas[i]

Ara ja tenim tots els valors que ens podrien interessar de la base de dades en valors numèrics. Fem una comprovació que això es compleix mirant-ne 10 entrades aleatòries:

In [9]:
dataset.sample(10)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
67287,Hip-Hop,Russ,Serious,2qfWkO1ZRcSdeHb1Cb3xIi,70,0.132,0.690,188947,0.635,0.000,1,0.362,-7.624,1,0.340,144.065,3,0.569
200018,Soundtrack,Kyle Dixon & Michael Stein,Lamps,0Fe4PvGPm4t6cpNGHrssuH,37,0.982,0.235,75013,0.062,0.949,1,0.144,-27.190,0,0.038,73.238,2,0.033
171515,Comedy,Steve Hofstetter,The Police Car is Half Full,7MHjx5NICkv3TF0kfrdd98,21,0.834,0.459,247375,0.619,0.000,10,0.383,-18.724,0,0.935,65.624,3,0.202
4001,Country,George Strait,You Can't Make A Heart Love Somebody - Edit,5CyXtO0ST8Uji8ArjuJsEU,38,0.140,0.560,168293,0.368,0.000,7,0.113,-9.706,1,0.028,138.293,3,0.195
141267,Indie,The Kooks,Pamela - Acoustic,1kdkQKFl1zPlk5Hwf1RdZQ,57,0.617,0.526,191753,0.638,0.000,7,0.090,-5.162,1,0.044,170.179,3,0.798
84498,R&B,New Edition,You're Not My Kind Of Girl,202QpSJo7inyEKWeARd5Ax,47,0.391,0.665,241693,0.787,0.000,11,0.554,-10.563,0,0.034,95.531,3,0.949
46112,Blues,Greta Van Fleet,"Lover, Leaver (Taker, Believer)",1AH28vdykcFi8iqPD1TCpg,53,0.001,0.505,361173,0.892,0.329,4,0.117,-4.244,0,0.038,91.999,3,0.548
181774,Movie,Auli'i Cravalho,How Far I'll Go - Reprise,32pqqCrnIC5O5iE9c218PJ,63,0.797,0.314,87347,0.441,0.000,1,0.130,-8.396,0,0.041,166.374,3,0.256
21656,Dance,The Wanted,Heartbreak Story,2Rj1Pt2TBkT9y8focbob5K,43,0.809,0.592,178827,0.355,0.000,11,0.131,-6.958,1,0.033,129.502,3,0.201
205491,Soundtrack,Bear McCreary,Wind and Rain,7M13r0yY8YdyPHn6Lc3Me9,26,0.469,0.353,206000,0.357,0.898,7,0.107,-15.778,1,0.035,67.381,3,0.079


Abans de continuar, aprofitarem per a normalitzar les nostres dades i eliminar les columnes inicials referents al gènere, el nom del artista, el nom de la cançó i la seva id, ja que no treballarem amb cap d'aquests valors.

In [10]:
index_dataset = dataset.drop(columns = ['genre', 'artist_name', 'track_name', 'track_id'])

In [11]:
normalitzat = (index_dataset - index_dataset.mean()) / index_dataset.std()
normalitzat.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,-2.265,0.687,-0.895,-1.469,1.287,-0.488,-1.185,0.663,1.291,0.731,-0.367,1.594,0.258,1.380
1,-2.210,-0.343,0.190,-1.048,0.629,-0.488,0.217,-0.322,0.668,-1.368,-0.182,1.822,0.258,1.387
2,-2.100,1.649,0.583,-0.682,-1.674,-0.488,-1.466,-0.565,-0.723,-1.368,-0.456,-0.590,2.756,-0.337
3,-2.265,0.946,-1.699,-0.880,-0.933,-0.488,-1.185,-0.587,-0.438,0.731,-0.438,1.749,0.258,-0.879
4,-2.045,1.643,-1.208,-1.655,-1.317,-0.081,-0.063,-0.065,-1.938,0.731,-0.405,0.740,0.258,-0.252


A partir d'aqui, podem visualitzar la matriu de correlació entre els diferentys atributs de la base de dades:

In [12]:
normalitzat.corr()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
popularity,1.000,-0.380,0.256,0.040,0.248,-0.211,-0.168,0.363,-0.149,0.080,0.059
acousticness,-0.380,1.000,-0.363,-0.022,-0.725,0.315,0.069,-0.689,0.150,-0.237,-0.324
danceability,0.256,-0.363,1.000,-0.137,0.324,-0.362,-0.042,0.437,0.134,0.020,0.546
duration_ms,0.040,-0.022,-0.137,1.000,-0.014,0.077,0.016,-0.027,-0.063,-0.013,-0.168
energy,0.248,-0.725,0.324,-0.014,1.000,-0.377,0.193,0.816,0.146,0.228,0.436
instrumentalness,-0.211,0.315,-0.362,0.077,-0.377,1.000,-0.134,-0.505,-0.176,-0.103,-0.306
liveness,-0.168,0.069,-0.042,0.016,0.193,-0.134,1.000,0.046,0.510,-0.051,0.012
loudness,0.363,-0.689,0.437,-0.027,0.816,-0.505,0.046,1.000,-0.002,0.227,0.398
speechiness,-0.149,0.150,0.134,-0.063,0.146,-0.176,0.510,-0.002,1.000,-0.081,0.023
tempo,0.080,-0.237,0.020,-0.013,0.228,-0.103,-0.051,0.227,-0.081,1.000,0.134
